# Find common syntax/word structures
We're going to look for common syntax structures among Linear A tablets. We classify each word/logogram
(including word separators) and come up with a list of tablets with common patterns.

First we load a lookup table of word types for classifying each word. We also have some helper functions

In [224]:
import csv                                                                                      
import json
import os
import re
from collections import Counter

def isNumber(word):
    if word >= u'\U00010100' and word <= u'\U0001013f':
        return True
    return False

place_names = {}
transaction_words = {}
transaction_signs = {}
numbers = {}
commodities = {}
from_suffix = {}
adjectives = {}
to_prefix = {}
fractions = {}
weights = {}
logograms = {}
words_in_linearb = {}
word_types = [(place_names, "060-place-names.txt", "place name", 0, 1),
        (transaction_words, "060-transaction-words.txt", "transaction term", 0, 1),
        (numbers, "050-numbers.csv", "number", 0, 1),
        (fractions, "065-fractions.txt", "fraction", 0, 0),
        (numbers, "065-fractions.txt", "number", 0, 0),
        (weights, "065-weights.txt", "weight", 0, 0),
        (logograms, "065-logograms.txt", "logogram", 0, 0),
        (commodities, "060-commodities.txt", "commodity", 0, 1),
        (from_suffix, "060-from-suffix.txt", "uses from suffix (-TE/-TI)", 0, 1),
        (adjectives, "060-adjectives.txt", "adjective for placename", 0, 1),
        (to_prefix, "060-to-prefix.txt", "uses to prefix (I-/J-)", 0, 1),
        (words_in_linearb, "135-identical-words-in-linearb.txt", "word also in linear b", 0, 1),
        (transaction_signs, "160-transaction-signs.txt", "transaction sign", 0, 1),
        ]
for word_type in word_types:
    dictionary = word_type[0]
    input_file = open("../" + word_type[1], 'r')
    while True:
        line = input_file.readline()
        if not line:
            break
        line_array = line.strip().split('\t')
        dictionary[line_array[word_type[3]]] = line_array[word_type[4]]

json_file = open('../150-metadata-template.js')
data = json.load(json_file)
inscriptions = data["metadata"]

def isHeadWord(word, original_word, no_of_words):
    if no_of_words == 1:
        return False
    if index:
        return False
    if original_word.startswith(u'\U0001076b'):
        return False
    if original_word == u'\U00010101':
        return False
    if word == "—":
        return False
    if word.isnumeric():
        return False
    return True;

def assignNumberToPreviousWord(word, word_tags, index, prev_word_tag, prev_original_word):
    if word not in numbers:
        return False
    if not index:
        return False
    if word == u'\U00010101':
        return False
    if word.startswith(u'\U0001076b'):
        return False
    cleaned_prev_original_word = prev_original_word.replace(u'\U0001076b', "")
    if cleaned_prev_original_word in numbers:
        return False
    if prev_original_word.endswith(u'\U0001076b'):
        return False
    if prev_original_word == "\n":
        return False
    return True

def shouldIncludeWord(word):
    if word in numbers:
        return False
    if u'\U00010101' in word:
        return False
    if word == "—":
        return False
    if word == "":
        return False
    return True

def wordRepeatedInInscription(word_tags, word):
    if not shouldIncludeWord(word):
        return False

    words = list(map(lambda x: x["word"].replace(u'\U0001076b', ""), word_tags))
    if words.count(word) > 1:
        return True
    return False

word_find_spots = {}
def addFindSpot(name, word):
    if not shouldIncludeWord(word):
        return
    find_spot = name[:2]
    if word not in word_find_spots:
        word_find_spots[word] = [find_spot]
        return
    word_find_spots[word].append(find_spot)

for inscription in inscriptions:
    for tags in inscription["tagsForWords"]:
        word = tags["word"]
        word = word.replace(u'\U0001076b', "")
        addFindSpot(inscription["name"], word)

locations = {}
locations["ZA"] ="Zakros"          
locations["PK"] ="Palaikastro"  
locations["PE"] ="Petras"  
locations["SY"] ="Syme" 
locations["PS"] ="Pseira" 
locations["MA" ] ="Malia" 
locations["AR"] ="Arkhalkhori" 
locations["IO"] ="Iouktas" 
locations["KN"] ="Knossos" 
locations["TY"] ="Tylissos"  
locations["PH"] ="Phaistos"  
locations["HT"] ="Haghia Triada"  
locations["AP"] ="Apodoulou"  
locations["KH"] ="Kharnia" 



## Create a list of all word structures

First create a list of word structures keyed by tablet name. Each word has a number of possible classifications so the word structure is a list of lists, for example the word structure for HT1 might look like:
```
'HT1': [['word'],
  ['word separator'],
  ['transaction term', 'transaction sign', 'word'],
  ['number'],
  ['word'],
  ['number'],
  ['word'],
  ['number'],
  ['word'],
  ['number'],
  ['word'],
  ['number']]
```

In [225]:
# Create a dictionary data structure containing word structure lists for all tablets.

word_structures = {}
new_inscriptions = []
for old_inscription in inscriptions:
    inscription = old_inscription.copy()
    word_tags = inscription["tagsForWords"]
    
    inscription["transactions"] = {}
    current_pattern = []
    
    for index, word_tag in enumerate(word_tags):
        word = word_tag["transliteratedWord"]
        original_word = word_tag["word"]
        if "tags" in word_tag:
            del word_tag["tags"]

        tags = []
        if word == "\n":
            continue

        if u'\U00010101' in original_word:
            tags.append("word separator")
        if original_word == "—":
            tags.append("dividing line")

        cleaned_word = word.replace(u'\U0001076b', "")
        cleaned_original_word = original_word.replace(u'\U0001076b', "")
        for word_type in word_types:
            dictionary = word_type[0]
            annotation = word_type[2]
            if cleaned_word in dictionary or cleaned_original_word in dictionary:
                if not annotation in tags:
                    tags.append(annotation)

        if (len(cleaned_word) > 1 and not "word separator" in tags
            and word != '—' and word !=  u'\U0001076b'
            and not 'logogram' in tags
            and not 'commodity' in tags
            and not "number" in tags and not "fraction" in tags):
            tags.append("word")

        if original_word == u'\U0001076b':
            tags.append("lacuna")
        else:
            for i, tag in enumerate(tags):
                if original_word.startswith(u'\U0001076b'):
                    tags[i] = "]" + tags[i]
                if original_word.endswith(u'\U0001076b'):
                    tags[i] = tags[i] + "["

        current_pattern += [sorted(tags)]
  
    inscription["words"] = inscription.pop("tagsForWords")
    word_structures[inscription["name"]] = current_pattern
    


Now that we have an exhaustive list of word structures, we strip out some of the word-tags so that we don't have too many permutations. We create `stripped_word_structures` for this.

In [226]:
import itertools

# To reduce the number of permutations we strip out word descriptions that are too specific.
def containsRedundantWord(l,
    words_to_drop = ["lacuna at", "transaction", "fraction", "weight", "commodity"]
    ):
    for w in words_to_drop:
        if w in l:
            return True
    return False

stripped_word_structures = {k: [[w for w in wl if not containsRedundantWord(w)]
                                for wl in v] 
                            for k,v in word_structures.items()}

def hasLacuna(l):
    return containsRedundantWord(l, ["lacuna"])

Create a list of word-tag permutations for each tablet. This will give us a list of different word-sequences for the tablet. We will use this to look for common sequences among all the tablets.

In [227]:
#small_sws = dict(list(stripped_word_structures.items())[0: 840])
#[list(x) for x in itertools.product(*stripped_word_structures['HT2'])]

# Create permutations for each tablet. It's a list of lists for each tablet.
ws_perms = {k: [list(x) for x in itertools.product(*v)]
             for k,v in stripped_word_structures.items()}
#dict(list(ws_perms.items())[0:2])

# Check for entries with too many permutations.
for k,v in ws_perms.items():
    if len(v) < 10:
        continue
    print(k, len(v))


Get all common sequences among the tablets and store it in `matched_sequences`.

In [254]:
%%time
from difflib import SequenceMatcher
from collections import defaultdict
from IPython.display import display, clear_output

"""To compare each tablet with every other tablet we take a copy of ws_perms and
use that as the one to check each tablet against. We delete each tablet from
the copy as we use it so that we're not comparing tablets more than once."""
ws_perms_copy = ws_perms.copy()
matched_sequences = defaultdict(list)
for k,v in ws_perms.items():
    # Delete from the copy so we don't perform unnecessary comparisons.
    del ws_perms_copy[k]
    # Check each permutation..
    for l in v:
        clear_output(wait=True)
        display(str(len(ws_perms) - len(ws_perms_copy)) + ' done.')
        s = SequenceMatcher(None)
        s.set_seq1(l)
        # .. against each permutation in the rest of the list.
        for k1,v1 in ws_perms_copy.items():
            for l1 in v1:
                s.set_seq2(l1)
                r = s.get_matching_blocks()
                for m in r:
                    # Ignore any sequences shorter than 3.
                    if m.size < 3:
                        continue
                    structure = ','.join(l[m.a:m.a+m.size])
                    matched_sequences[structure] += [(k,m.a,m.size), (k1,m.b,m.size)]
                    matched_sequences[structure] = list(set(matched_sequences[structure]))


'1712 done.'

CPU times: user 33.7 s, sys: 2.09 s, total: 35.8 s
Wall time: 39.9 s


Now we have enough data to print out matching sequences. We're going to start by looking at sequences at the start of inscriptions.

In [259]:
def getSyntacticalStructures(matched_sequences
                             , from_start_only=False
                             , min_length=3
                             , min_results=3):
    """Get all instances of common syntatical structure, either `from_start_only` or not, with at least"
    `min_results` matches of `min_length`"""
    # Create a list of words in each tablet that we can look up by tablet name.
    la_words = {inscription["name"] : [v["word"] for v in inscription["tagsForWords"]
                                       if v["word"] != '\n'] 
                for inscription in inscriptions}
    en_words = {inscription["name"] : [v["transliteratedWord"] for v in inscription["tagsForWords"]
                                       if v["word"] != '\n'] 
                for inscription in inscriptions}

    structures = {}
    # We're looking for sequences of 3 words or more at the start of the inscription.
    for i in range(50,min_length,-1):
        test_seq = {k:v for k,v in matched_sequences.items() if len(k.split(',')) == i}
        for k,v in test_seq.items():
            en_seqs = {}
            for tablet, start, end in v:
                # We're only looking at structures at the start of the inscription.
                if start and from_start_only:
                    continue
                en_seqs[tablet] = en_words[tablet][start:start+end]

            # Ignore any cases where we don't have `min_results` inscriptions with a matching sequence.
            if len(en_seqs.keys()) < min_results:
                continue

            structures[k] = en_seqs
    return structures



In [260]:
import pandas as pd
styles = [dict(selector="caption", 
    props=[("text-align", "center"),
    ("font-size", "120%"),
    ("color", 'black')])]

syn_seqs = getSyntacticalStructures(matched_sequences, True)
for k, en_seqs in syn_seqs.items():
    df = pd.DataFrame(en_seqs.values(), columns = k.split(','))
    df.set_axis(en_seqs.keys(), axis='index', inplace=True)
    #df.style.set_caption("Davis 2018 Mapping of Linear A and PD Symbols").set_table_styles(styles)
    display(df)

,word,word separator,word,word separator,logogram,number,logogram,number,logogram,number
HT21,PI-TA-KA-SE,𐄁,TE,𐄁,GRA,161,OLE+U,11,OLE+E,3
HT28a,A-SI-JA-KA,𐄁,JA-*21F,𐄁,GRA+QE,5,OLE+U,2,OLE+KI,¹⁄₂
HT14,PU-VIN,𐄁,TE,𐄁,GRA,30,OLE+MI,3,OLE+DI,3


,word,number,word,number,word,number,word,number
ZA14,ME-KI-DI,1,*21F-*118,1,PU-NI-KA-*363,3,QA-TI-JU,8
HT99b,SI+SE-NI,6,DA-SI-*118,2,ZU-DU,1,RU-MA-TA,1
PH2,A-SE-TU-*21F,1,RA-O-DI-KI,60,PI-RU-E-JU,60,SE-SA-PA₃,60
HT10b,U-TI,1,DA-RE,2,TA-RI-NA,15,*312-TA,6


,word,word separator,word,word separator,logogram,number,word,number
HT19,RA-*164-TI,𐄁,TE,𐄁,VIN,30,SA-RO,5
KH88,QA-NU-MA,𐄁,*21F-*118,𐄁,NI,10,PU-DE,8
HT17,RA-󽇫-TI,𐄁,TE,𐄁,VIN,37,SA-RO,10
ZA8,KI-RA,𐄁,A-TA-RE,𐄁,NI,¹⁄₂,KU-TU-KO-RE,double mina


,word,word separator,word,word separator,logogram,number,logogram,number
HT21,PI-TA-KA-SE,𐄁,TE,𐄁,GRA,161,OLE+U,11
HT28a,A-SI-JA-KA,𐄁,JA-*21F,𐄁,GRA+QE,5,OLE+U,2
HT14,PU-VIN,𐄁,TE,𐄁,GRA,30,OLE+MI,3
HT92,TE,𐄁,A-DU,𐄁,GRA,680,*304,12


,word,word separator,word,number,word,number,word
HT85b,KI-KI-RA-JA,𐄁,KI-RE-TA₂,1,QE-KA,1,PA
HT94b,KI-RO,𐄁,TU-MA,1,PA-TA-NE,1,DE-DI
HT1,QE-RA₂-U,𐄁,KI-RO,197,ZU-SU,70,DI-DI-ZA-KE


...

,lacuna,]logogram,number[,]logogram
KH63,𐝫,SU,10,*401+RU
KH84,𐝫,CYP,¹⁄₂,*301
HT131b,𐝫,NI,30,OLIV
KH26,𐝫,VIR+[?],60,NI


,]logogram[,number,logogram,number
MA6a,HIDE+[?],941,HIDE+B,35
KH12,CYP,2,*308,¹⁄₂
KH15,CYP,¹⁄₂,NI,¹⁄₂


,lacuna,lacuna,]word[,lacuna
HT154ja,𐝫,𐝫,NU-TE,𐝫
KH52,𐝫,𐝫,TA-U,𐝫
KH44,𐝫,𐝫,*335,𐝫


,]logogram,word separator,word,word separator
KH14,RA,𐄁,A-MA-JA,𐄁
PLZf1,TA,𐄁,WI-TE-JA-MU,𐄁
KNZb4,JU,𐄁,JA-SI,𐄁
PKZa10,SI,𐄁,I-PI-NA-MI-NA,𐄁


,]logogram,number[,]logogram,number[
THEtab.4,CAP,7,CAP,46
KH25,VIR+[?],140,VIR+[?],10
PH3b,]MI+JA,³⁄₄,SI,¹⁄₄


In [261]:
syn_seqs = getSyntacticalStructures(matched_sequences,False,8)
for k, en_seqs in syn_seqs.items():
    df = pd.DataFrame(en_seqs.values(), columns = k.split(','))
    df.set_axis(en_seqs.keys(), axis='index', inplace=True)
    #df.style.set_caption("Davis 2018 Mapping of Linear A and PD Symbols").set_table_styles(styles)
    display(df)

,word,word separator,word,number,word,number,word,number,word,number,word,number,word,number,word,number
HT117a,U-MI-NA-SI,𐄁,U-SU,1,MI-TU,1,KU-RA-MU,1,MA-RU,1,KU-PA₃-NU,1,TU-JU-MA,1,U-DI-MI,1
HT88,KI-RO,𐄁,KU-PA₃-PA₃,1,KA-JU,1,KU-PA₃-NU,1,PA-JA-RE,1,SA-MA-RO,1,DA-TA-RE,1,KU-RO,6
HT9b,KA-*305,𐄁,PA-DE,3,A-SI,3,*306-TU,8,*324-DI-RA,2,QE-PU,2,TA-I-AROM,2,DI-NA-U,4


,word,number,word,number,word,number,word,number,word,number,word,number,word,number,word,number
HT122a,TE-KI,2,QA-*310-I,3,JA-MI-DA-RE,1,SI-DA-RE,1,*324-DI-RA,1,PA-DE,1,KU-PA₃-NU,1,PA-TA-NE,1
HT9b,PA-DE,3,A-SI,3,*306-TU,8,*324-DI-RA,2,QE-PU,2,TA-I-AROM,2,DI-NA-U,4,KU-RO,24
HT117a,MI-TU,1,KU-RA-MU,1,MA-RU,1,KU-PA₃-NU,1,TU-JU-MA,1,U-DI-MI,1,MI-RU-TA-RA-RE,1,TE-JA-RE,1


,word separator,word,number,word,number,word,number,word,number,word,number,word,number,word,number
HT117a,𐄁,U-SU,1,MI-TU,1,KU-RA-MU,1,MA-RU,1,KU-PA₃-NU,1,TU-JU-MA,1,U-DI-MI,1
HT88,𐄁,KU-PA₃-PA₃,1,KA-JU,1,KU-PA₃-NU,1,PA-JA-RE,1,SA-MA-RO,1,DA-TA-RE,1,KU-RO,6
HT85a,𐄁,DA-RI-DA,12,PA₃-NI,12,U-*325-ZA,6,DA-SI-*118,24,KU-ZU-NI,5,TE-KE,3,DA-RE,4
HT9b,𐄁,PA-DE,3,A-SI,3,*306-TU,8,*324-DI-RA,2,QE-PU,2,TA-I-AROM,2,DI-NA-U,4


,number,word,number,word,number,word,number,word,number,word,number,word,number,word,number
HT9b,3,A-SI,3,*306-TU,8,*324-DI-RA,2,QE-PU,2,TA-I-AROM,2,DI-NA-U,4,KU-RO,24
HT117a,1,MI-TU,1,KU-RA-MU,1,MA-RU,1,KU-PA₃-NU,1,TU-JU-MA,1,U-DI-MI,1,MI-RU-TA-RA-RE,1
HT122a,1,TE-KI,2,QA-*310-I,3,JA-MI-DA-RE,1,SI-DA-RE,1,*324-DI-RA,1,PA-DE,1,KU-PA₃-NU,1


,logogram,number,logogram,number,logogram,number,logogram,number,word,logogram,number,logogram,number,logogram,number
KH11,VIN,≈ ¹⁄₆,*306,4,CYP,¹⁄₃,*348-CYP,𐝇𐝉,A-TO-*349-TO-I,CYP+E,3,NI,1,VIN,3
HT28a,OLE+U,2,OLE+KI,¹⁄₂,OLE+MI,1,OLE+TU,𐝉,SA-RA₂,OLE+DI,1,NI,2,VIN,3
HT101,GRA+QE,40,OLE+U,8,OLE+MI,8,OLE+KI,2,SA-RA₂,GRA,41,OLE,10,DI+QE,3
HT14,OLE+MI,3,OLE+DI,3,*304,9,OLIV,13,A-PU₂-NA-DU,GRA,45,OLE+MI,5,OLE+DI,4


...

,number,logogram,number,word,number,word,number,logogram,number
HT23a,10,E,17,*21F-RI-TU-QA,¹⁄₁₆,SA-SA-ME,¹⁄₁₆,SI+ME,10
HT89,24,VIR+*313a,13,TA-RA,5,KU-RO,87,CYP,2
HT49a,1,*312,1,KU-PA₃-NU,1,TU-SU-PU₂,1,KI,1


,word,word separator,word,word separator,word,logogram,number,logogram,number
HT28b,A-SI-JA-KA,𐄁,U-MI-NA-SI,𐄁,SA-RA₂,GRA,20,OLE+DI,5
KH5,A-DA-KI-SI-KA,𐄁,A-RA-U-DA,𐄁,WI-SA-SA-NE,CYP+E,2,VINb+WI,2
KH7a,I-JA-PA-ME,𐄁,TA-TA,𐄁,QA-TI-KI,VIR+*313b,4,CYP+D,¹⁄₃


,number,word,number,logogram,number,word,number,word,number
HT119,10,KU-PA₃-NA-TU,7,TI,7,JA-*345,10,*306-TU,2
HT118,4,A-RI-SU,4,KI,1,RI-RU-MA,10,KU-RO,30
HT89,22,MA-I-MI,24,VIR+*313a,13,TA-RA,5,KU-RO,87


,word,word separator,word,word separator,word,word separator,word,word separator,word
KNZf31,DA-DU-MI-NE,𐄁,QA-MI-*47-NA-RA,𐄁,A-WA-PI,𐄁,TE-SU-DE-SE-KE-I,𐄁,A-DA-RA
PLZf1,WI-TE-JA-MU,𐄁,U-QE-TI,𐄁,JA-SA-SA-RA-ME,𐄁,TA-NU-NI-KI-NA,𐄁,NI-NU-NI
CR(?)Zf1,A-MA-WA-SI,𐄁,KA-NI-JA-MI,𐄁,I-JA,𐄁,QA-KI-SE-NU-TI,𐄁,A-TA-DE
KOZa1,TU-RU-SA,𐄁,DU-*314-RE,𐄁,I-DA-A,𐄁,U-NA-KA-NA-SI,𐄁,I-PI-NA-MA


,word,number,word,number,word,number,word,number,lacuna
PH2,A-SE-TU-*21F,1,RA-O-DI-KI,60,PI-RU-E-JU,60,SE-SA-PA₃,60,𐝫
ZA20,SI-TU,6,TE-AROM,12,RU-MA-TA-SE,3,KU-RA,130,𐝫
PE2,A-RI-PA,¹⁄₄,QA-QA-DA,¹⁄₂,TO-ME,¹⁄₃,TO-*49-RE,1,𐝫


In [280]:
mat_seq_cnt = {k: len(v) for k,v in matched_sequences.items()
               if len(k.split(',')) > 8 and len(v) > 5}

df = pd.DataFrame(mat_seq_cnt.values(), columns = ["Count"])
df.set_axis(mat_seq_cnt.keys(), axis='index', inplace=True)
#df.style.set_caption("Davis 2018 Mapping of Linear A and PD Symbols").set_table_styles(styles)
display(df.sort_values(by=['Count'], ascending=False))



,Count
"number,word,number,word,number,word,number,word,number",29
"word,number,word,number,word,number,word,number,word,number",23
"number,word,number,word,number,word,number,word,number,word,number",14
"logogram,number,logogram,number,logogram,number,logogram,number,word",13
"word,number,word,number,word,number,word,number,word,number,word",12
"word,number,word,number,word,number,word,number,word,number,word,number",11
"logogram,number,logogram,number,logogram,number,logogram,number,logogram,number",10
"word separator,word,number,word,number,word,number,word,number,word,number",8
"word,word separator,word,number,word,number,word,number,word,number,word,number",7
"number,word,number,word,number,word,number,word,number,word,number,word,number",7


In [229]:
word_structures["HT24a"]

[['word'],
 ['logogram['],
 [']number'],
 ['logogram'],
 ['number'],
 [']number'],
 ['fraction', 'number'],
 ['logogram'],
 ['number'],
 ['adjective for placename', 'word'],
 ['logogram['],
 ['word'],
 ['logogram[']]

In [248]:
{20} & {20, 10} 

{20}

In [18]:
# Write out a template file for the selected inscription
name = "HT86a"
output_file = open(name + ".js", "w")
output_file.write(json.dumps([x for x in inscriptions if x["name"] == name], sort_keys=True, indent=4, ensure_ascii=False))
output_file.close()


